# Using REPLUG for Parallel Document Reading with LLMs

In this tutorial, we will demostrate how we can utilize the REPLUG algorithm to read multiple documents in parallel to generate an answer for any question.

We will use a pipeline that will:

- Fetch relevant documents for our question.
- Rerank the documents for better performance.
- Read in parallel and generate according to REPLUG.

For more information about REPLUG, we encourage reading the original paper: [REPLUG: Retrieval-Augmented Black-Box Language Models
](https://arxiv.org/abs/2301.12652). 

Below we showcase a comparison of the CODEX model when using REPLUG on MMLU, as specified in Table 2 of the original paper:

| Model | Parameters | Humanities | Social. | STEM | Other | All |
| --- | --- | --- | --- | --- | --- | --- |
| Codex | 175B | 74.2 | 76.9 | 57.8 | 70.1 | 68.3 |
| Codex + REPLUG | 175B | 76.0 | 79.7 | 58.8 | 72.1 | 71.4

We observe that the REPLUG approach performs better across all categories


In [ ]:
from haystack import Pipeline
from haystack.nodes.prompt import PromptNode
import torch
from haystack.nodes import PromptModel
from haystack.nodes.prompt.prompt_template import PromptTemplate
from haystack.nodes import AnswerParser
from haystack.nodes.ranker import SentenceTransformersRanker
from haystack.nodes.retriever import BM25Retriever
from haystack.document_stores import InMemoryDocumentStore
from haystack import Document

We start from a collection of paragraphs from Wikipedia, for the retrieval phase:

In [2]:
document_collection = [{'id': '11457596',
  'text': 'Quest", the "Ultima" series, "EverQuest", the "Warcraft" series, and the "Elder Scrolls" series of games as well as video games set in Middle-earth itself. Research also suggests that some consumers of fantasy games derive their motivation from trying to create an epic fantasy narrative which is influenced by "The Lord of the Rings". In 1965, songwriter Donald Swann, who was best known for his collaboration with Michael Flanders as Flanders & Swann, set six poems from "The Lord of the Rings" and one from "The Adventures of Tom Bombadil" ("Errantry") to music. When Swann met with Tolkien to play the',
  'title': 'The Lord of the Rings'},
 {'id': '11457582',
  'text': 'helped "The Lord of the Rings" become immensely popular in the United States in the 1960s. The book has remained so ever since, ranking as one of the most popular works of fiction of the twentieth century, judged by both sales and reader surveys. In the 2003 "Big Read" survey conducted in Britain by the BBC, "The Lord of the Rings" was found to be the "Nation\'s best-loved book". In similar 2004 polls both Germany and Australia also found "The Lord of the Rings" to be their favourite book. In a 1999 poll of Amazon.com customers, "The Lord of the',
  'title': 'The Lord of the Rings'},
 {'id': '11457540',
  'text': 'of Tolkien\'s works is such that the use of the words "Tolkienian" and "Tolkienesque" has been recorded in the "Oxford English Dictionary". The enduring popularity of "The Lord of the Rings" has led to numerous references in popular culture, the founding of many societies by fans of Tolkien\'s works, and the publication of many books about Tolkien and his works. "The Lord of the Rings" has inspired, and continues to inspire, artwork, music, films and television, video games, board games, and subsequent literature. Award-winning adaptations of "The Lord of the Rings" have been made for radio, theatre, and film. In',
  'title': 'The Lord of the Rings'},
 {'id': '11457587',
  'text': 'has been read as fitting the model of Joseph Campbell\'s "monomyth". "The Lord of the Rings" has been adapted for film, radio and stage. The book has been adapted for radio four times. In 1955 and 1956, the BBC broadcast "The Lord of the Rings", a 13-part radio adaptation of the story. In the 1960s radio station WBAI produced a short radio adaptation. A 1979 dramatization of "The Lord of the Rings" was broadcast in the United States and subsequently issued on tape and CD. In 1981, the BBC broadcast "The Lord of the Rings", a new dramatization in 26',
  'title': 'The Lord of the Rings'},
 {'id': '11457592',
  'text': '"The Lord of the Rings", was released on the internet in May 2009 and has been covered in major media. "Born of Hope", written by Paula DiSante, directed by Kate Madison, and released in December 2009, is a fan film based upon the appendices of "The Lord of the Rings". In November 2017, Amazon acquired the global television rights to "The Lord of the Rings", committing to a multi-season television series. The series will not be a direct adaptation of the books, but will instead introduce new stories that are set before "The Fellowship of the Ring". Amazon said the',
  'title': 'The Lord of the Rings'},
 {'id': '7733817',
  'text': 'The Lord of the Rings Online The Lord of the Rings Online: Shadows of Angmar is a massive multiplayer online role-playing game (MMORPG) for Microsoft Windows and OS X set in a fantasy universe based upon J. R. R. Tolkien\'s Middle-earth writings, taking place during the time period of "The Lord of the Rings". It launched in North America, Australia, Japan, and Europe in 2007. Originally subscription-based, it is free-to-play, with a paid VIP subscription available that provides players various perks.  The game\'s environment is based on "The Lord of the Rings" and "The Hobbit". However, Turbine does not',
  'title': 'The Lord of the Rings Online'},
 {'id': '22198847',
  'text': 'of "The Lord of the Rings", including Ian McKellen, Andy Serkis, Hugo Weaving, Elijah Wood, Ian Holm, Christopher Lee, Cate Blanchett and Orlando Bloom who reprised their roles. Although the "Hobbit" films were even more commercially successful than "The Lord of the Rings", they received mixed reviews from critics. Numerous video games were released to supplement the film series. They include: "," Pinball, "", "", , "", "", "", "", "The Lord of the Rings Online", "", "", "", "Lego The Lord of the Rings", "Guardians of Middle-earth", "", and "".',
  'title': 'The Lord of the Rings (film series)'},
 {'id': '24071573',
  'text': 'Lord of the Rings (musical) The Lord of the Rings is the most prominent of several theatre adaptations of J. R. R. Tolkien\'s epic high fantasy novel of the same name, with music by A. R. Rahman, Christopher Nightingale and the band Värttinä, and book and lyrics by Matthew Warchus and Shaun McKenna. Set in the world of Middle-earth, "The Lord of the Rings" tells the tale of a humble hobbit who is asked to play the hero and undertake a treacherous mission to destroy an evil, magic ring without being seduced by its power. The show was first performed',
  'title': 'Lord of the Rings (musical)'},
 {'id': '11457536',
  'text': 'The Lord of the Rings The Lord of the Rings is an epic high fantasy novel written by English author and scholar J. R. R. Tolkien. The story began as a sequel to Tolkien\'s 1937 fantasy novel "The Hobbit", but eventually developed into a much larger work. Written in stages between 1937 and 1949, "The Lord of the Rings" is one of the best-selling novels ever written, with over 150 million copies sold. The title of the novel refers to the story\'s main antagonist, the Dark Lord Sauron, who had in an earlier age created the One Ring to rule',
  'title': 'The Lord of the Rings'},
 {'id': '13304003',
  'text': "The Lord of the Rings (disambiguation) The Lord of the Rings is a fantasy novel by J. R. R. Tolkien. The title refers to Sauron, the story's main antagonist. The Lord of the Rings may also refer to:",
  'title': 'The Lord of the Rings (disambiguation)'}]

We then create an InMemoryDocumentStore document store, to store all the documents:

In [3]:
store = InMemoryDocumentStore(use_bm25=True)

In [4]:
documents = [Document(id=item["id"], content=item["text"], meta={"title": item["title"]}) for item in document_collection]

In [5]:
store.write_documents(documents)

Updating BM25 representation...: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 28397.45 docs/s]


Next, we create a simple BM25 retriever on top of our store, and an additional reranker component to improve the ranking of the documents used for answering the question:

In [ ]:
retriever = BM25Retriever(
    document_store= store,
    top_k= 10
)

reranker = SentenceTransformersRanker(
    batch_size= 32,
    model_name_or_path= "cross-encoder/ms-marco-MiniLM-L-6-v2",
    top_k= 1,
    use_gpu= False
)

Now that we have created the retrieval components, we move to the LLM usage.

To use REPLUG, we first create a document template that contains a placeholder for the retrieved documents to be inserted.
Given that the model receives $K$ documents, the component will generate $K$ prompts for each document, using the current template.
The ###REPLUG-DOC### place holder is where the retrieved document to be inserted. 

We also insert the query itself, to instruct the model to answer the question, given the current document.

In [7]:
from typing import Dict, Any

def remove_template_params(self, kwargs: Dict[str, Any]) -> Dict[str, Any]:
    return kwargs
PromptTemplate.remove_template_params = remove_template_params

AParser = AnswerParser()
LFQA = PromptTemplate(
    prompt="""[INST] <<SYS>>
Answer the Question below using only the Document provided.
Do not use any prior knowledge to answer the question.
Your answer can only be an entity name or a short phrase.

Examples:
Document: "The Lamentable and Tragical History of Titus Andronicus," also called "Titus Andronicus\' Complaint," is a ballad from the 17th century about the fictional Roman general, Titus, and his revenge cycle with the Queen of the Goths. Events in the ballad take place near the end of the Roman Empire, and the narrative of the ballad parallels the plot of William Shakespeare\'s play Titus Andronicus.
Question: Shakespeare's "Titus Andronicus" is set during the latter days of which Empire?
Answer: Roman Empire

Document: Popeye the Sailor Man is a fictional cartoon character created by Elzie Crisler Segar. The character first appeared on January 17, 1929, in the daily King Features comic strip Thimble Theatre.
Question: Who created Popeye?
Answer: Elzie Segar

Document: he Rose Period (Spanish: Período rosa) comprises the works produced by Spanish painter Pablo Picasso between 1904 and 1906. It began when Picasso settled in Montmartre at the Bateau-Lavoir among bohemian poets and writers.
Question: What period preceded Picasso's "Rose Period"?
Answer: Blue Period

Document: ###REPLUG-DOC###
<</SYS>>

Question: {query}
Answer: [/INST]""",
    output_parser= AParser
)

We now create a PromptModel with an LLM, using the ```code replug.ReplugHFLocalInvocationLayer``` class, to manage the parallel prompting done by REPLUG.

In [8]:
from fastrag.prompters.invocation_layers.replug import ReplugHFLocalInvocationLayer

In [9]:
PrompterModel = PromptModel(
    model_name_or_path= "meta-llama/Llama-2-7b-chat-hf",
    use_gpu= True,
    invocation_layer_class=ReplugHFLocalInvocationLayer,
    model_kwargs= dict(
        max_new_tokens=10,
        model_kwargs= dict(
            device_map = "auto",
            torch_dtype = torch.bfloat16),
        generation_kwargs=dict(do_sample=True)))

[12/17/2023 18:06:43] {utils.py:137} INFO - Using devices: CUDA:0 - Number of GPUs: 1
Loading checkpoint shards: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:07<00:00,  3.77s/it]
The model 'HF' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJFor

With the model and the prompt template now ready, we create a PromptNode to unify both modules:

In [10]:
Prompter = PromptNode(
    model_name_or_path= PrompterModel,
    default_prompt_template= LFQA
)

Our components are now ready. We can now create a pipeline, to connect all of them together:

In [11]:
pipe = Pipeline()

pipe.add_node(component=retriever, name= 'Retriever',inputs= ["Query"])
pipe.add_node(component=reranker, name= 'Reranker',inputs= ["Retriever"])
pipe.add_node(component=Prompter, name= 'Prompter',inputs= ["Reranker"])

Finally, lets ask it a question:

In [14]:
answer_result = pipe.run("Who is the main villan in Lord of the Rings?",params={
    "Retriever": {
        "top_k": 8
    },
    "Reranker": {
        "top_k": 5
    }
})

In [15]:
print(answer_result["answers"][0].answer)

 Answer: Sauron
